In [ ]:
from selenium import webdriver
import time
from openpyxl import Workbook
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

import warnings 
warnings.filterwarnings('ignore')

wb = Workbook(write_only=True)
ws = wb.create_sheet()

driver = webdriver.Chrome("chromedriver.exe")
driver.get("https://www.youtube.com/watch?v=8pSC6QgxFzI&t=593s")
driver.implicitly_wait(3)

time.sleep(1.5)

driver.execute_script("window.scrollTo(0, 800)")
time.sleep(3)

last_height = driver.execute_script("return document.documentElement.scrollHeight")

#while True: # 끝까지 내릴때
for i in range(0,2): # 2번만 내리겠다.
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(1.5)

    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    if new_height == last_height: 
        break
    last_height = new_height

time.sleep(3)

# 유튜브 프리미엄 팝업이 뜨는 것을 닫아주는 역할
try:
    driver.find_element(By.CSS_SELECTOR,"#dismiss-button > a").click()
except:
    pass

html_source = driver.page_source
soup = BeautifulSoup(html_source, 'html.parser')

                        #div중에 id가 header-author인걸 찾고 id가 autor-text인걸 찾고, span부분
id_list = soup.select("div#header-author > h3 > #author-text > span")

                        #yt-formatted-string 요소 중에 id가 content-text인것들 찾기
comment_list = soup.select("yt-formatted-string#content-text")

print(comment_list)

id_final = []
comment_final = []


for i in range(len(comment_list)):
    temp_id = id_list[i].text
    
    #데이터 확인
    #print(id_list[i])
    #print(id_list[i].text)
    
    temp_id = temp_id.replace('\n', '')
    temp_id = temp_id.replace('\t', '')
    temp_id = temp_id.replace('    ', '')
    id_final.append(temp_id) # 댓글 작성자

    temp_comment = comment_list[i].text
    temp_comment = temp_comment.replace('\n', '')
    temp_comment = temp_comment.replace('\t', '')
    temp_comment = temp_comment.replace('    ', '')
    comment_final.append(temp_comment) # 댓글 내용

pd_data = {"아이디" : id_final , "댓글 내용" : comment_final}
youtube_pd = pd.DataFrame(pd_data)

youtube_pd.to_excel('Youtube_result.xlsx')